In [1]:
import networkx as nx
import numpy as np

import scipy.optimize
import scipy.sparse

from networkx.utils import np_random_state

from src.python._process_params import _process_params
from src.python.getMatrix import getMatrixByName
from src.python.vis.visGraph import visGraph

In [2]:
# Copied from networkx.drawing.layout.py
@np_random_state(10)
def spring_layout(
    G,
    k=None,
    pos=None,
    fixed=None,
    iterations=50,
    threshold=1e-4,
    weight="weight",
    scale=1,
    center=None,
    dim=2,
    seed=None,
    method="FR",  # ! added method = "FR" or "RS"
):
    G, center = _process_params(G, center, dim)

    if fixed is not None:
        if pos is None:
            raise ValueError("nodes are fixed without positions given")
        for node in fixed:
            if node not in pos:
                raise ValueError("nodes are fixed without positions given")
        nfixed = {node: i for i, node in enumerate(G)}
        fixed = np.asarray([nfixed[node] for node in fixed if node in nfixed])

    if pos is not None:
        # Determine size of existing domain to adjust initial positions
        dom_size = max(coord for pos_tup in pos.values() for coord in pos_tup)
        if dom_size == 0:
            dom_size = 1
        pos_arr = seed.rand(len(G), dim) * dom_size + center

        for i, n in enumerate(G):
            if n in pos:
                pos_arr[i] = np.asarray(pos[n])
    else:
        pos_arr = None
        dom_size = 1

    if len(G) == 0:
        return {}
    if len(G) == 1:
        return {nx.utils.arbitrary_element(G.nodes()): center}

    # ! Changed a lot
    A = nx.to_scipy_sparse_array(G, weight=weight, dtype="f")
    if k is None and fixed is not None:
        # We must adjust k by domain size for layouts not near 1x1
        nnodes, _ = A.shape
        k = dom_size / np.sqrt(nnodes)
    return _sparse_fruchterman_reingold(
        A, k, pos_arr, fixed, iterations, threshold, dim, seed, method
    )

In [3]:
from networkx.utils import np_random_state
from src.python.cost import cost
import scipy as sp
from typing import Tuple


# Copied from networkx.drawing.layout.py
@np_random_state(7)
def _sparse_fruchterman_reingold(
    A,
    k=None,
    pos=None,
    fixed=None,
    iterations=50,
    threshold=1e-4,
    dim=2,
    seed=None,
    method="FR",
    verbose=True,
):
    from scipy.optimize import minimize

    try:
        nnodes, _ = A.shape
    except AttributeError as err:
        msg = "fruchterman_reingold() takes an adjacency matrix as input"
        raise nx.NetworkXError(msg) from err

    if pos is None:
        # random initial positions
        pos = np.asarray(seed.rand(nnodes, dim), dtype=A.dtype)
    else:
        # make sure positions are of same type as matrix
        pos = pos.astype(A.dtype)

    # no fixed nodes
    if fixed is None:
        fixed = []

    # optimal distance between nodes
    if k is None:
        k = np.sqrt(1.0 / nnodes)

    if method == "FR":
        # make sure we have a LIst of Lists representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        # the initial "temperature" is about .1 of domain area (=1x1)
        # this is the largest step allowed in the dynamics.
        t = max(max(pos.T[0]) - min(pos.T[0]), max(pos.T[1]) - min(pos.T[1])) * 0.1
        # simple cooling scheme.
        # linearly step down by dt on each iteration so last iteration is size dt.
        dt = t / (iterations + 1)

        displacement = np.zeros((dim, nnodes))
        for iteration in range(iterations):
            displacement *= 0
            # loop over rows
            for i in range(A.shape[0]):
                if i in fixed:
                    continue
                # difference between this row's node position and all others
                delta = (pos[i] - pos).T
                # distance between points
                distance = np.sqrt((delta**2).sum(axis=0))
                # enforce minimum distance of 0.01
                distance = np.where(distance < 0.01, 0.01, distance)
                # the adjacency matrix row
                Ai = A.getrowview(i).toarray()  # TODO: revisit w/ sparse 1D container
                # displacement "force"
                displacement[:, i] += (
                    delta * (k * k / distance**2 - Ai * distance / k)
                ).sum(axis=1)
            # update positions
            length = np.sqrt((displacement**2).sum(axis=0))
            length = np.where(length < 0.01, 0.1, length)
            delta_pos = (displacement * t / length).T
            pos += delta_pos
            # cool temperature
            t -= dt
            if verbose:
                print(f"{cost(pos,A,k)=}")
            if (np.linalg.norm(delta_pos) / nnodes) < threshold:
                break
            yield pos
    elif method == "L-BFGS-B" or method == "BFGS" or method == "CG":
        # make sure we have a coo_matrix representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        k_inv = 1 / k

        def cost_fun(x):
            pos = x.reshape((nnodes, dim))
            grad = np.zeros((nnodes, dim))
            for i in range(nnodes):
                delta = pos[i] - pos
                assert np.all(delta[i] == 0.0)
                distance = np.linalg.norm(delta, axis=1)
                distance = np.where(distance < 0.01, 0.01, distance)
                distance_inv = 1 / distance
                Ai = A.getrow(i).toarray().flatten()
                coefficient1 = Ai * distance * k_inv - (k * distance_inv) ** 2
                grad[i] = coefficient1 @ delta
            ret = cost(pos, A, k)
            print(f"{ret=}")
            return ret, grad.ravel()

        pos_hist = []
        res = sp.optimize.minimize(
            cost_fun,
            pos.ravel(),
            method=method,
            jac=True,
            options={"maxiter": iterations, "disp": verbose},
            callback=lambda x: pos_hist.append(x.reshape((nnodes, dim))),
        )

        for pos in pos_hist:
            yield pos

        if verbose:
            print("         WarnFlag and message: %d" % res.status, res.message)
            print("         Current function value: %f" % res.fun)
            print("         Iterations: %d" % res.nit)

    elif method == "myCG":
        from scipy.optimize._optimize import _line_search_wolfe12

        # make sure we have a coo_matrix representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        k_inv = 1 / k

        def f(x: np.ndarray) -> float:
            pos = x.reshape((nnodes, dim))
            ret = cost(pos, A, k)
            print(f"{ret=}")
            return ret

        def myfprime(x: np.ndarray) -> np.ndarray:
            pos = x.reshape((nnodes, dim))
            grad = np.zeros((nnodes, dim))
            for i in range(nnodes):
                delta = pos[i] - pos
                assert np.all(delta[i] == 0.0)
                distance = np.linalg.norm(delta, axis=1)
                distance = np.where(distance < 0.01, 0.01, distance)
                distance_inv = 1 / distance
                Ai = A.getrow(i).toarray().flatten()
                coefficient1 = Ai * distance * k_inv - (k * distance_inv) ** 2
                grad[i] = coefficient1 @ delta
            return grad.ravel()

        x0 = pos.ravel()
        gtol = 1e-5
        disp = True
        c1 = 1e-4
        c2 = 0.4

        old_fval = f(x0)
        gfk = myfprime(x0)

        k_cg = 0
        xk = x0
        old_old_fval = old_fval + np.linalg.norm(gfk) / 2

        warnflag = 0
        pk = -gfk
        gnorm = np.amax(np.abs(gfk))

        sigma_3 = 0.01

        while (gnorm > gtol) and (k_cg < iterations):
            deltak = np.dot(gfk, gfk)

            cached_step = [None]

            def polak_ribiere_powell_step(alpha, gfkp1=None):
                xkp1 = xk + alpha * pk
                if gfkp1 is None:
                    gfkp1 = myfprime(xkp1)
                yk = gfkp1 - gfk
                beta_k = max(0, np.dot(yk, gfkp1) / deltak)
                pkp1 = -gfkp1 + beta_k * pk
                gnorm = np.amax(np.abs(gfkp1))
                return (alpha, xkp1, pkp1, gfkp1, gnorm)

            def descent_condition(alpha, xkp1, fp1, gfkp1):
                # Polak-Ribiere+ needs an explicit check of a sufficient
                # descent condition, which is not guaranteed by strong Wolfe.
                #
                # See Gilbert & Nocedal, "Global convergence properties of
                # conjugate gradient methods for optimization",
                # SIAM J. Optimization 2, 21 (1992).
                cached_step[:] = polak_ribiere_powell_step(alpha, gfkp1)
                alpha, _, pk, gfk, gnorm = cached_step

                # Accept step if it leads to convergence.
                if gnorm <= gtol:
                    return True

                # Accept step if sufficient descent condition applies.
                return np.dot(pk, gfk) <= -sigma_3 * np.dot(gfk, gfk)

            try:
                alpha_k, _, _, old_fval, old_old_fval, gfkp1 = _line_search_wolfe12(
                    f,
                    myfprime,
                    xk,
                    pk,
                    gfk,
                    old_fval,
                    old_old_fval,
                    c1=c1,
                    c2=c2,
                    amin=1e-100,
                    amax=1e100,
                    extra_condition=descent_condition,
                )
                print(f"{alpha_k=}")
            except RuntimeError:
                # Line search failed to find a better solution.
                warnflag = 2
                break

            # Reuse already computed results if possible
            if alpha_k == cached_step[0]:
                alpha_k, xk, pk, gfk, gnorm = cached_step
            else:
                alpha_k, xk, pk, gfk, gnorm = polak_ribiere_powell_step(alpha_k, gfkp1)

            k_cg += 1

            yield xk.reshape((nnodes, dim))

        fval = old_fval
        if disp:
            if warnflag == 2:
                msg = "pr_loss"
            elif k_cg >= iterations:
                warnflag = 1
                msg = "maxiter"
            elif np.isnan(gnorm) or np.isnan(fval) or np.isnan(xk).any():
                warnflag = 3
                msg = "nan"
            else:
                msg = "success"
            print("         WarnFlag and message: %d" % warnflag, msg)
            print("         Current function value: %f" % fval)
            print("         Iterations: %d" % k)

    elif method == "CGVR":
        from scipy.optimize._optimize import _line_search_wolfe12

        # make sure we have a coo_matrix representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        disp = True
        gtol = 1e-5
        c1 = 1e-4
        c2 = 0.4
        c3 = 0.01
        xk = pos

        cache_f = [None]

        def f_i(xk_i: np.ndarray, i: int) -> Tuple[float, np.ndarray]:
            assert xk_i.shape == (dim,)
            if np.all(xk_i == cache_f[0]):
                return cache_f[1:]
            delta = xk_i - xk
            delta[i].fill(0.0)
            distance = np.linalg.norm(delta, axis=1)
            distance = np.where(distance < 0.01, 0.01, distance)
            Ai = A.getrow(i).toarray().flatten()
            EPS = 1e-10
            val = np.sum(Ai * (distance**3) / (3 * k) - (k**2) * np.log(distance + EPS))
            grad = (Ai * distance / k - (k / distance) ** 2) @ delta
            cache_f[:] = (xk_i, val, grad)
            return cache_f[1:]

        gfk = [np.zeros(dim) for _ in range(nnodes)]
        old_fval = [0.0 for _ in range(nnodes)]
        old_old_fval = [0.0 for _ in range(nnodes)]
        for i in range(nnodes):
            old_fval[i], gfk[i] = f_i(xk[i])
            old_old_fval[i] = old_fval[i] + np.linalg.norm(gfk[i]) / 2

        warnflag = 0

        k_cg = 0
        while k_cg < iterations:
            uk = [np.zeros(dim) for _ in range(nnodes)]
            for i in range(nnodes):
                uk[i] = f_i(xk[i])[1]
            xk_0 = np.copy(xk)
            p = [-gfk_i for gfk_i in gfk]
            gnorm = max(np.amax(np.abs(gfk_i)) for gfk_i in gfk)
            if gnorm <= gtol:
                break

            deltak = sum(np.dot(gfk_i, gfk_i) for gfk_i in gfk)

            for i in range(nnodes):
                cached_step = [None]

                def polak_ribiere_powell_step(alpha, gfkp1_i=None):
                    xkp1_i = xk[i] + alpha * pk[i]
                    if gfkp1_i is None:
                        gfkp1_i = f_i(xkp1_i)[1]
                    yk = gfkp1_i - gfk[i]
                    beta_k = max(0, np.dot(yk, gfkp1_i) / deltak)
                    pkp1 = -gfkp1_i + beta_k * pk
                    gnorm = np.amax(np.abs(gfkp1_i))
                    return (alpha, xkp1, pkp1, gfkp1_i, gnorm)

                def descent_condition(alpha, _xkp1, _fp1, gfkp1):
                    # Polak-Ribiere+ needs an explicit check of a sufficient
                    # descent condition, which is not guaranteed by strong Wolfe.
                    #
                    # See Gilbert & Nocedal, "Global convergence properties of
                    # conjugate gradient methods for optimization",
                    # SIAM J. Optimization 2, 21 (1992).
                    cached_step[:] = polak_ribiere_powell_step(alpha, gfkp1)
                    alpha, _, pk, gfk, gnorm = cached_step

                    # Accept step if it leads to convergence.
                    if gnorm <= gtol:
                        return True

                    # Accept step if sufficient descent condition applies.
                    return np.dot(pk, gfk) <= -c3 * np.dot(gfk, gfk)

                try:
                    alpha_k, _, _, old_fval[i], old_old_fval[i], gfkp1 = (
                        _line_search_wolfe12(
                            lambda xk_i: f_i(xk_i, i)[0],
                            lambda xk_i: f_i(xk_i, i)[1],
                            xk[i],
                            pk[i],
                            gfk[i],
                            old_fval[i],
                            old_old_fval[i],
                            c1=c1,
                            c2=c2,
                            amin=1e-100,
                            amax=1e100,
                            extra_condition=descent_condition,
                        )
                    )
                    print(f"{alpha_k=}")
                except RuntimeError:
                    # Line search failed to find a better solution.
                    warnflag = 2
                    break

                # Reuse already computed results if possible
                if alpha_k == cached_step[0]:
                    alpha_k, xk, pk, gfk, gnorm = cached_step
                else:
                    alpha_k, xk, pk, gfk, gnorm = polak_ribiere_powell_step(
                        alpha_k, gfkp1
                    )

            k_cg += 1

            yield xk.reshape((nnodes, dim))

        fval = old_fval
        if disp:
            if warnflag == 2:
                msg = "pr_loss"
            elif k_cg >= iterations:
                warnflag = 1
                msg = "maxiter"
            elif np.isnan(gnorm) or np.isnan(fval) or np.isnan(xk).any():
                warnflag = 3
                msg = "nan"
            else:
                msg = "success"
            print("         WarnFlag and message: %d" % warnflag, msg)
            print("         Current function value: %f" % fval)
            print("         Iterations: %d" % k)

    else:
        raise ValueError()

In [56]:
mat = getMatrixByName("jagmesh1")

if scipy.sparse.issparse(mat):
    mat.setdiag(0)
    mat.eliminate_zeros()
    mat.data = np.abs(mat.data)
else:
    mat[np.diag_indices_from(mat)] = 0
    mat.data = np.abs(mat.data)

G = nx.Graph(mat)


if False:
    for iteration, pos in enumerate(spring_layout(G, method="FR", iterations=50)):
        if iteration % 5 == 0:
            visGraph(G, pos)
    # visGraph(G, list(spring_layout(G, method="FR", iterations=250))[-1])
if True:
    import time

    t0 = time.perf_counter()
    # list(spring_layout(G, seed=0, method="L-BFGS-B", iterations=10))
    t1 = time.perf_counter()
    # list(spring_layout(G, seed=0, method="CG", iterations=10))
    t2 = time.perf_counter()
    list(spring_layout(G, seed=0, method="myCG", iterations=10))
    t3 = time.perf_counter()

    print(f"L-BFGS-B: {t1-t0}[sec]")
    print(f"      CG: {t2-t1}[sec]")
    print(f"    myCG: {t3-t2}[sec]")

ret=np.float64(14284.042775097787)
ret=np.float64(11100.294607333897)
ret=np.float64(3813.8346665761064)
alpha_k=np.float64(0.0029043966788128827)
ret=np.float64(390221.1810293812)
ret=np.float64(3059.8415855778567)
ret=np.float64(31275.174184878557)
ret=np.float64(2083.4173716333544)
alpha_k=np.float64(0.005788957819632335)
ret=np.float64(9340.421186670099)
ret=np.float64(1818.4737750036486)
alpha_k=np.float64(0.010606575794178803)
ret=np.float64(1754.8779343736119)
ret=np.float64(1559.8307267640448)
alpha_k=np.float64(0.05516197837260731)
ret=np.float64(2129.5763473535376)
ret=np.float64(1414.3100378916083)
ret=np.float64(1460.5048392960923)
ret=np.float64(1376.2595692643135)
alpha_k=np.float64(0.038502182616530406)
ret=np.float64(1754.0724968769764)
ret=np.float64(1255.8258464643966)
ret=np.float64(1249.2859036411355)
ret=np.float64(1288.8284333207805)
ret=np.float64(1227.3160717880842)
alpha_k=np.float64(0.025728142461905176)
ret=np.float64(1244.9887249014794)
ret=np.float64(1081.0